<a href="https://colab.research.google.com/github/Karn2898/GenAI_Learning_Archive/blob/main/ChromaDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install chromadb openai langchain tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currentl

In [ ]:
!pip show chromadb

Name: chromadb
Version: 1.5.0
Summary: Chroma.
Home-page: https://github.com/chroma-core/chroma
Author: 
Author-email: Jeff Huber <jeff@trychroma.com>, Anton Troynikov <anton@trychroma.com>
License: 
Location: /usr/local/lib/python3.12/dist-packages
Requires: bcrypt, build, grpcio, httpx, importlib-resources, jsonschema, kubernetes, mmh3, numpy, onnxruntime, opentelemetry-api, opentelemetry-exporter-otlp-proto-grpc, opentelemetry-sdk, orjson, overrides, posthog, pybase64, pydantic, pypika, pyyaml, rich, tenacity, tokenizers, tqdm, typer, typing-extensions, uvicorn
Required-by: 


In [ ]:
!wget -q https://github.com/selva86/datasets/blob/master/sample.txt

In [ ]:
!unzip -q sample.txt

[sample.txt]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of sample.txt or
        sample.txt.zip, and cannot find sample.txt.ZIP, period.


setting up env

In [ ]:
import os
os.environ['OPENAI_API_KEY']=''

In [ ]:
!pip install langchain

In [ ]:

import langchain
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

In [ ]:
DirectoryLoader("path to dir", glob="./txt",loader_cl=TextLoader )

In [ ]:
document=loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
text_splitter= RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text=text_splitter.split_documents(document)

In [ ]:
len(text)

In [ ]:
text[1]

creating DB

In [ ]:
from langchain import embeddings
persist_dir='db'
embeddding=OpenAIEmbeddings()
vectordb=chroma.from_documents(documents=text,embedding=embeddding,persist_directory=persist_dir)

In [ ]:
vectordb.persist()
vectordb=None

In [ ]:
vectordb=chroma(persist_directiry =persist_dir,
                embedding_function=embedding)

make retriever

In [ ]:
retriever=vectordb.as_retriever()

In [ ]:
retriever.get_relevant_documents("how much money did ms raise?")

In [ ]:
len(docs)

In [ ]:
retriever=vectordb.as_retriever(search_type="similarity", search_kwargs={"k":2})

In [ ]:
retrirver.search_type

In [ ]:
retriever.search_kwargs

make a chain

In [ ]:
from langchain.chains import RetrievalQA
qa_chain=RetrievalQA.from_chain_type(llm=OpenAI(),
                                     chain_types="stuff",
                                     retrirver=retriever,
                                     return_source_documents=True)

In [ ]:
def process_llm_responses(llm_response):
  print(llm_response['result'])
  print('\n\nSources:')
  for source in llm_response["source_documents"]:
    print(source.metadata['source'])

In [ ]:
query="how much did ms raise?"
llm_response=qa_chain(query)
llm_response
